<a href="https://colab.research.google.com/github/sbbzplt/QiskitFallFest_IZTECH/blob/master/Grovers_maxcut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Grover's algorithm to solve the Max Cut problem

We will try solving a famous problem called the Maximum Cut Problem (or Max Cut in short) using Grover's algorithm.

In [ ]:
from qiskit import *

## The Max Cut Problem

The Maximum Cut problem, or Max Cut in short, is a problem involving separating the vertex set of an undirected graph into two zones in order to maximize the number of edges crossing the two zones. Here we denote any undirected graph as G, vertices as V and edges as E. The two groups when separated by a line (or a cut) will be separate zones.

Given this notation, we need to find out how to maximize the number of edges E that crosses between the two zones.

### Solving the Max Cut problem using a quantum algorithm
There are several approaches to solving a Max Cut problem using a quantum algorithm such as:  
[Quantum Approximate Optimization Algorithm (QAOA)](https://qiskit.org/documentation/aqua/optimization/qiskit_optimization.html) and 
[Variational Quantum Eigensolver (VQE)](https://medium.com/qiskit/the-variational-quantum-eigensolver-43f7718c2747).

Here we are going to use Grover's algorithm.

Let us start with a simple example where we have
(0,1,2）as our three vertices and
(0,1),(1,2) as our two edges connecting these vertices.
<img src="maxcut.png" width="320">


Let us cut this graph to create two zones in three different ways. (The edges crossing different zones will be called **'cut-edges'** from here on.)
As you can see here, the first cut pattern gives us the maximum value of edges crossing different zones.
And there you have it. The maximum value of cut-edges for this graph is 2.

### Judging the cut-edges using a quantum circuit
Let us first build a judgement circuit to determine if we have a cut-edge or not in a graph.

We assign qubits to each vertex and then assign 0 and 1 to each subgraph (zone).   

Example: Say we have two vertices. We will use 1 qubit for each vertex.

-00 means both vertices are in zone 0
-01 means first vertex is in zone 0 and second vertex is in zone 1
-10 means first vertex is in zone 1 and second vertex is in zone 0
-11 means both vertices are in zone 1

When the vertices (a, b) are in the same zone, then the oracle will give us 0. 
When the vertices (a, b) are in a different group or zone, then the oracle will give us 1.  
Look at the truth table below. Notice that the results are the same as an XOR circuit.

|Vertex a|Vertex b|Result|
|--|--|--|
|0|0|0|
|0|1|1|
|1|0|1|
|1|1|0|

Let us look at what the corresponding circuit looks like.   


In [ ]:
#cutedge checker
#Creates a circuit with n qubits, checks edge ab, and stores the result in c
def ccheck(n, a, b, c):
    qc = QuantumCircuit(n)
    qc.cx(a,c) 
    qc.cx(b,c)
    return qc

qc = ccheck(3, 0,1,2)
qc.draw(output='mpl')

## Counting the cut-edges

We have learnt how to count the number of 1s in a state. 

Let's look at the graph above again. It has 3 vertices and 2 edges.

For our subgraph containing its vertices in q[0],q[1],q[2], we will encode the judgement results to q[3],q[4].

Then by counting the number of 1s stored in q[3] and q[4], we will get the number of cut-edges and store this in q[5],q[6].

In [ ]:
qc = QuantumCircuit(8)

#Check edge 01, store in qubit 3
qc += ccheck(8,0,1,3)
#Check edge 12, store in qubit 4
qc += ccheck(8,1,2,4)

qc.barrier()

#add qubit 3
qc.cx(3,5)

#add qubit 4
qc.ccx(4,5,6)
qc.cx(4,5)

qc.draw(output='mpl')

Now we have a circuit that counts the cut-edges. 


## Creating an Oracle for the Max Cut Problem
Recall from the “Grover’s Search Algorithm” we learned in week 2 that the marking operation is completely determined by the Boolean oracle function f, which takes a single quantum state as an input and spits out whether or not the state is a search target. This means, we simply need to create a circuit that will either say 'Yes' or 'No' according to the input.

Let's say we want to create an oracle that can determine whether the cut-edges is 2.

We can check if 2 i.e. 10 is stored in qubits numbered 5 and 6 and set qubit 7 to 1 if this is the case.

In [ ]:
qc = QuantumCircuit(8,1)

#Let's assign vertices to zones to check if the oracle is working correct or not

#101 (vertex 0 in zone 1, vertex 1 in zone 0, vertex 2 in zone 1)
qc.x(0)
qc.x(2)

#Check edge 01, store in qubit 3
qc += ccheck(8,0,1,3)
#Check edge 12, store in qubit 4
qc += ccheck(8,1,2,4)

qc.barrier()

#add qubit 3
qc.cx(3,5)

#add qubit 4
qc.ccx(4,5,6)
qc.cx(4,5)

#Check if qubits 5 and 6 store 10 (note that it will be in reverse order 01 i.e. qubit 5:0 qubit6:1)
qc.x(5)
qc.ccx(5,6,7)
qc.x(5)

qc.measure(7,0)

backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=100000)
result = job.result()
count =result.get_counts()
print(count)
qc.draw(output='mpl')

From the results, we can see that qubit 7 is measured 1, hence the initial assignment of vertices to zones we have checked yields 2 cut edges.

After operating the phase flip, we reverse all operations we have applied so far to bring q[3] to q[6] back to their initial states. 
This operation is performed to eliminate unnecessary entanglements between the input qubits q[0] to q[2] and oracle qubits.

# Completing our circuit for the Max Cut Problem
By adding the Diffusion circuit (a.k.a. amplitude amplification) we learned in the previous week, we can complete our circuit for our Max Cut Problem.
Let's try and see if we can obtain the right answer  010 and 101 with high probability.

In [ ]:
def inversion_about_average():
    """Apply inversion about the average step of Grover's algorithm."""
    circuit = QuantumCircuit(8)
    for i in range(3):
        circuit.h(i)
    for i in range(3):
        circuit.x(i)
    circuit.h(2)
    circuit.ccx(0,1,2)
    circuit.h(2)
    for i in range(3):
        circuit.x(i)
    for i in range(3):
        circuit.h(i)
    return circuit

In [ ]:
def oracle():
    qc = QuantumCircuit(8)
    #Check edge 01, store in qubit 3
    qc += ccheck(8,0,1,3)
    #Check edge 12, store in qubit 4
    qc += ccheck(8,1,2,4)

    qc.barrier()

    #add qubit 3
    qc.cx(3,5)

    #add qubit 4
    qc.ccx(4,5,6)
    qc.cx(4,5)

    #Check if qubits 5 and 6 store 10 and save this in qubit 7 (note that it will be in reverse order 01 i.e. qubit 5:0 qubit6:1)
    qc.x(5)
    qc.ccx(5,6,7)
    qc.x(5)
    
    return qc

In [ ]:
groverCircuit = QuantumCircuit(8,3)
for i in range(3):
    groverCircuit.h(i)

#query phase
groverCircuit += oracle()
#note that we store the result in qubit 7
groverCircuit.z(7)
groverCircuit += oracle().inverse()

groverCircuit.barrier()

#inversion phase
groverCircuit += inversion_about_average()

#measure in reverse order
for i in range(3):    
    groverCircuit.measure(i,2-i)

backend = BasicAer.get_backend('qasm_simulator')
shots = 1024
results = execute(groverCircuit, backend=backend, shots=shots).result()
answer = results.get_counts()
print(answer)

We've successfully obtained our answers.

### Task 1

Decide if the given graph has **3 cut-edges** where vertices V and edges E are:<br/>
V: (0,1,2,3)<br/>
E: (0,1),(0,2),(0,3)  
by using Grover's Algorithm with **iteration = 2** times.

Use qubits 0,1,2,3 for the vertices, qubits 4,5,6 to store the result of edge checks, 7 and 8 to store the count and 9 as output.

Complete the oracle function.

In [ ]:
#cutedge checker
#Creates a circuit with n qubits, checks edge ab, and stores the result in c
def ccheck(n, a, b, c):
    qc = QuantumCircuit(n)
    qc.cx(a,c) 
    qc.cx(b,c)
    return qc

In [ ]:
def oracle():
    qc = QuantumCircuit(10)

    #Your code here
    
    return qc

In [ ]:
def inversion_about_average():
    """Apply inversion about the average step of Grover's algorithm."""
    circuit = QuantumCircuit(10)
    for i in range(4):
        circuit.h(i)
    for i in range(4):
        circuit.x(i)
    circuit.h(3)
    circuit.mct([0,1,2],3)
    circuit.h(3)
    for i in range(4):
        circuit.x(i)
    for i in range(4):
        circuit.h(i)
    return circuit

In [ ]:
groverCircuit = QuantumCircuit(10,4)
for i in range(4):
    groverCircuit.h(i)

iterations = 2
for i in range(iterations):
    #query phase
    groverCircuit += oracle()
    #note that we store the result in qubit 9
    groverCircuit.z(9)
    groverCircuit += oracle().inverse()

    groverCircuit.barrier()

    #inversion phase
    groverCircuit += inversion_about_average()

#measure in reverse order
for i in range(4):    
    groverCircuit.measure(i,3-i)

backend = BasicAer.get_backend('qasm_simulator')
shots = 1024
results = execute(groverCircuit, backend=backend, shots=shots).result()
answer = results.get_counts()
print(answer)

[click for our solution](Grovers_maxcut_solutions.ipynb#task1)